In [1]:
!pip install folium

   ---------------------------------------- 0.0/108.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/108.9 kB ? eta -:--:--
   ------------------------------ --------- 81.9/108.9 kB 1.2 MB/s eta 0:00:01
   ------------------------------------- -- 102.4/108.9 kB 1.5 MB/s eta 0:00:01
   -------------------------------------- 108.9/108.9 kB 905.1 kB/s eta 0:00:00


In [2]:
!pip install openpyxl

In [47]:
#문제 1번 전국의 대학교 위치 시각화

import pandas as pd

filePath = r'고등교육기관 주소록(2024.4.1.).xlsx'
df_from_excel = pd.read_excel(filePath,engine='openpyxl')
df_from_excel.columns = df_from_excel.loc[4].tolist()
df_from_excel = df_from_excel.drop(index=list(range(0,5)))

print(df_from_excel.head())

print(df_from_excel['학교명'].values)

print(df_from_excel['주소'].values)

     연도   학교종류  시도     행정구                 학교명  \
5  2024    대학교  강원  강원 원주시           국립강릉원주대학교   
6  2024    대학교  강원  강원 강릉시           국립강릉원주대학교   
7  2024  일반대학원  강원  강원 원주시        국립강릉원주대학교대학원   
8  2024  일반대학원  강원  강원 강릉시        국립강릉원주대학교대학원   
9  2024  특수대학원  강원  강원 원주시  국립강릉원주대학교경영정책과학대학원   

                                             학교명(영문)  KEDI학교코드        본분교  \
5                Gangneung-Wonju National University  51001000  본교(제2캠퍼스)   
6                Gangneung-Wonju National University  51001000  본교(제1캠퍼스)   
7  Gangneung-Wonju National University Graduate S...  51001600  본교(제2캠퍼스)   
8  Gangneung-Wonju National University Graduate S...  51001600  본교(제1캠퍼스)   
9  Gangneung Wonju Graduate School of Management ...  51001607  본교(제2캠퍼스)   

    학교상태  설립   우편번호                                       주소          전화번호  \
5  학교명변경  국립  26403  강원도 원주시 흥업면 남원로 150 (흥업리, 강릉원주대학교원주캠퍼스)  033-760-8114   
6  학교명변경  국립  25457             강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)  033-640-7001   
7 

In [46]:
# 전국의 대학교 위치 시각화
import requests

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'   #도로명주소
road_type2 = 'PARCEL' #지번주소
address = '&address='
keys = '&key='
primary_key = '2A645740-77BD-380B-81D8-8AB51C078AB8'

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y

x,y = request_geo("광주광역시 북구 하서로 85 (운암동, 한국폴리텍V대학)")

print(f'x값: {x}')
print(f'y값: {y}')

x값: 126.879618351
y값: 35.184234636


In [20]:
import pandas as pd
import requests
from openpyxl import load_workbook
from openpyxl import Workbook
import re


filePath = r'고등교육기관 주소록(2024.4.1.).xlsx'
df_from_excel = pd.read_excel(filePath,engine='openpyxl')
df_from_excel.columns = df_from_excel.loc[4].tolist()
df_from_excel = df_from_excel.drop(index=list(range(0,5)))


url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'   #도로명주소
road_type2 = 'PARCEL' #지번주소
address = '&address='
keys = '&key='
primary_key = '2A645740-77BD-380B-81D8-8AB51C078AB8'

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y


try:
    wb = load_workbook(r"학교주소좌표.xlsx", data_only=True)
    sheet  = wb.active
except:
    wb = Workbook()
    sheet = wb.active

university_list = df_from_excel['학교명'].to_list()
address_list = df_from_excel['주소'].to_list()

for num,value in enumerate(address_list):
    addr = re.sub(r'\([^)]*\)', '', value)
    print(addr)
    x,y = request_geo(addr)
    sheet.append([university_list[num],addr,x,y])

wb.save(r"학교주소좌표.xlsx")

강원도 원주시 흥업면 남원로 150 
강원도 강릉시 죽헌길 7 
강원도 원주시 흥업면 남원로 150 
강원도 강릉시 죽헌길 7 
강원도 원주시 흥업면 남원로 150 
강원도 강릉시 죽헌길 7 
강원도 원주시 흥업면 남원로 150 
강원도 강릉시 죽헌길 7 
강원도 원주시 흥업면 남원로 150 
강원도 강릉시 죽헌길 7 
강원도 춘천시 강원대학길 1 
강원도 삼척시 중앙로 346 
강원도 춘천시 강원대학길 1 
강원 삼척시 중앙로 346
강원도 춘천시 강원대학길 1 
강원도 춘천시 강원대학길 1 
강원도 삼척시 중앙로 346 
강원도 춘천시 강원대학길 1 
강원도 춘천시 강원대학길 1 
강원도 춘천시 강원대학길 1 
강원도 춘천시 강원대학길 1 
강원도 삼척시 중앙로 346 
강원도 춘천시 강원대학길 1 
대구 중구 국채보상로 680
대구광역시 북구 대학로 80 
경북 상주시 경상대로 2559
대구 중구 국채보상로 680
대구광역시 북구 대학로 80 
경북 상주시 경상대로 2559
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
대구 중구 국채보상로 680
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
경북 상주시 경상대로 2559
대구 북구 대학로 80
대구광역시 북구 대학로 80 
대구 중구 국채보상로 680
대구 중구 국채보상로 680
대구 중구 국채보상로 680
대구 중구 달구벌대로 2177
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
대구광역시 북구 대학로 80 
경남 창원시 의창구 차룡로48번길 54
경상남도 진주시 진주대로 501 
경남 통영시 통영해안로 2
경상남도 진주시 진주대로 501 
경남 통영시 통영해안로 2
경상남도 진주시 진주대로 501 
경상남도 진주시 진주대로 501 
경상남도 진주시 진주대로816번길 15 
경상남도 진주시 진주대로 501 
경상남도 진주시 진주대로816번길 15 
경상남도 진주시 동

In [49]:
# 문제 2번 한국폴리텍대학 광주캠퍼스의 위치에 마커를 표시하는 코드 만들기

import folium

map = folium.Map(location=[37,127],zoom_start=7)

marker = folium.Marker([35.182955197, 126.878670165],
                    popup='한국폴리텍V대학 광주캠퍼스', 
                    icon = folium.Icon(color='pink'))

marker.add_to(map) 

map.save(r'한국폴리텍대학 광주캠퍼스.html')

In [45]:
# 문제 3번 

import pandas as pd

filePath = r'지역별 폭염일수.xlsx'
df_from_excel = pd.read_excel(filePath,engine='openpyxl')
df_from_excel.columns = df_from_excel.loc[0].tolist()
df_from_excel = df_from_excel.drop(index=list(range(0)))

print(df_from_excel)

    순창군(254)   127.1286  35.37131   9
0   순창군(254)  127.12860  35.37131   9
1   진도군(268)  126.25846  34.47296   8
2    여수(168)  127.74063  34.73929   5
3    해남(261)  126.56907  34.55375   9
4    홍천(212)  127.88043  37.68360   2
..       ...        ...       ...  ..
86   거제(294)  128.60459  34.88818   4
87   대전(133)  127.37210  36.37199  11
88   산청(289)  127.87910  35.41300   6
89   보령(235)  126.55744  36.32724   9
90   영천(281)  128.95140  35.97742   8

[91 rows x 4 columns]


In [50]:
import requests

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'
road_type2 = 'PARCEL'
address = '&address='
keys = '&key='
primary_key = '2A645740-77BD-380B-81D8-8AB51C078AB8'

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y

x,y = request_geo("광주광역시 북구 하서로 85 (운암동, 한국폴리텍V대학)")

print(f'x값: {x}')
print(f'y값: {y}')

x값: 0
y값: 0
